In [ ]:
import apache_beam as beam
from apache_beam.transforms.window import{
    TimestampedValue,
    Sessioins,
    Duration,
}
from apache_beam.io.textio import WriteToText


class AddTimestampDoFn(beam.DoFn):
    def process(self, element):
        unix_timestamp = element["timestamp"]
        element = (element["user"], element["click"])
        
        yield TimestampedValue(element, unix_timestamp)
        
        
with beam.Pipeline() as p:
    # fmt: off
    events = p | beam.Create(
           [
            {"userID":"Andy", "click":1, "timestamp":1603112520}, # Event time: 13:02
            {"userID":"Sam",  "click":1, "timestamp":1603113240}, # Event time: 13:14
            {"userID":"Andy", "click":1, "timestamp":1603115820}, # Event time: 13:57
            {"userID":"Andy", "click":1, "timestamp":1603113600}, # Event time: 13:20
              
           ]
    )
    # fmt: on
    timestamped_events = events | beam.ParDo(AddTimestampDoFn())
    windowed_events = timestamped_events | beam.WindowInto(Sessions(gap_size = 30 * 60))
    sum_clicks = windowed_events | beam.CombinePerKey(sum)
    sum_clicks |WriteToText(file_path_prefix = "output")